# BILAG: WEBSCRAPING AF JOBINDEX

###### Note: For at følge med og reproducere analyserne, skal du ned hente Jupyter Notebook. Det kan gøres på https://jupyter.org/. Du kan køre celler nemt ved at trykke Ctrl + Enter i cellemarkeringen. Det er vigtigt at du gemmer den vedhæftede 'pickle'-fil (final_scraping.pkl) i samme mappe som denne notesbog.

Dette bilag vil redegøre for min dataindsamling og -behandling

Første af snit omhandler hvordan dataene er indsamlet. 

Herefter følges klargørelse og analyse.

###### Centralt er to strategier i min dataklargørelse: 
1. Sørge for, at der kan skabes et aggregeret overblik over ord på tværs af jobannoncer (med stadig mulighed for sektoradskillelse)
2. Sørge for, at hver jobannonce bevarer sin unikke kombination af ord


In [5]:
# Dette er de forskellige udviklerpakker, som jeg bruger undervejs. Skal køres. 
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import nltk
import string
import lemmy
import numpy
import sys

## Her skabes mit datasæt bestående af jobannoncer fra Jobindex 

### OBS: SKAL IKKE KØRES, KØR FRA "Her klargøres og oprydes i datasættet"

Dette behover ikke køres, såfremt du ønsker at indlæse min pickle-fil forneden. Dette indebærer, at du får lov til at anvende præcis den samme data som mig, fremfor at hente nyt ned. Derfor anbefales det, at dette afsnits celler ikke køres. 

In [2]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup

# index = []
# list_companies = []
# beskrivelse_liste = []
# antal_stjerner = []
# month_list = []
# year_list = []
# day_list = []
# url_liste = []
# category_job = []

# antal_sider = 1130

# for j in range(1, antal_sider + 1):
#     url = "https://www.jobindex.dk/jobsoegning/kontor/jura?jobage=archive&maxdate=20211121&mindate=20050101&page={}&q=jurist+advokat".format(j)
#     result = requests.get(url)
#     src = result.content
#     soup = BeautifulSoup(src, 'html.parser')
#     jobs = soup.find_all("div", {"class": 'PaidJob'})


#     # Kodning
    
#     for i, job in enumerate(jobs):
        
#         # Navnet på organisationen
#         try:
#             company = job.p.a.b.text
#             list_companies.append(company)
#         except:
#             list_companies.append('NaN')

#         # tekst fra kroppen af annoncen
#         text = ''
#         for p in job.find_all('p')[1:]:
#             text += p.text

#          # bulletpoints fra annoncen
#         opgaver_liste = ''
#         opgaver = job.find_all('li')


#         for opgave in opgaver:
#             if '\n' not in opgave.text:
                
#                 text += opgave.text

#         beskrivelse_liste.append(text)

#         try: 
#             stjerner = job.find('span', {'class': 'sr-only'}).text.split()[0]
#             antal_stjerner.append(stjerner)
#         except:
#             antal_stjerner.append('NaN')

#         # data
#         dato = str(job.find('time'))
#         year = dato[16:20]
#         month = dato[21:23]
#         day = dato[24:26]

#         month_list.append(month)
#         year_list.append(year)
#         day_list.append(day)

#         link = job.find_all('a')[6]['href']
#         url_liste.append(link)



        
#     time.sleep(2)
    


In [3]:
df = pd.DataFrame()

In [4]:
df['index'] = pd.Series(index)
df['company'] = pd.Series(list_companies)
df['jobbeskrivelse'] = pd.Series(beskrivelse_liste)
df['antal_stjerner'] = pd.Series(antal_stjerner)
df['month'] = pd.Series(month_list)
df['year'] = pd.Series(year_list)
df['day'] = pd.Series(day_list)
df['url'] = pd.Series(url_liste)

<ipython-input-4-68d20d33f58d>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['index'] = pd.Series(index)


In [5]:
df["company"].iloc[10000]

'BankInvest Gruppen'

In [6]:
df["jobbeskrivelse"].iloc[10000]

'\nVi søger en erfaren jurist til selvstændigt at give juridisk rådgivning til BankInvest Gruppens direktion og afdelinger og til at løse en række opgaver under overskriften ”legal risk management”, herunder arbejde med implementering af ny lovgivning. Du vil i vid udstrækning selv være med til at identificere relevante arbejdsopgaver. \nDu vil indgå som en del af Legal Department, som ud over en chefjurist består af tre jurister/advokater samt to sekretærer. '

In [7]:
df["year"].iloc[10000]

'2008'

In [324]:
df.to_pickle("final_scraping.pkl") # gemmer ovenstående dataframe til en pickle-fil

### START HERFRA:

## Her klargøres og opryddes datasættet

In [6]:
df = pd.read_pickle("final_scraping.pkl") # indlæser pickle-fil

df["company"]
df

,index,company,jobbeskrivelse,antal_stjerner,month,year,day,url,english,offentlig
0,NaN,Telenor,Kunne du tænke dig at arbejde i en intern juri...,4,11,2021,11,https://candidate.hr-manager.net/ApplicationIn...,0,0
1,NaN,Alm. Brand,Kan du håndtere regressager som blander forsik...,5,11,2021,09,https://candidate.hr-manager.net/ApplicationIn...,0,0
2,NaN,Herlev Kommune,"Herlev Kommune søger en jurist, der betegner s...",3,11,2021,08,https://herlev.emply.net/recruitment/vacancyAd...,0,1
3,NaN,Skattestyrelsen,Hvis du er klar på at hjælpe danske og udenlan...,4,11,2021,08,https://candidate.hr-manager.net/ApplicationIn...,0,1
4,NaN,PenSam,Har du lyst til at blive PenSam Banks jurist o...,NaN,11,2021,06,https://hr-portal.azets.dk/REK/Pensam/Joblist/...,0,0
...,...,...,...,...,...,...,...,...,...,...
10418,NaN,SkandiaBanken,\nDine primære ansvarsområder er:\nDu er uddan...,NaN,01,2007,15,http://www.jobindex.dk/annoncer/55588.shtml,0,0
10419,NaN,Realkredit Danmark,\nEr du erfaren jurist eller akademiker med an...,5,01,2007,12,http://www.rd.dk/,0,0
10420,NaN,BRFkredit,\nMed reference til den administrerende direkt...,NaN,01,2007,09,http://www.brf.dk/C12569D50035A140/alldocs/DOC...,0,0
10421,NaN,Dansk Retursystem A/S,"\nVi søger en jurist på nedsat tid, der i sama...",4,01,2007,08,http://www.danskretursystem.dk/composite-257.htm,0,0


#### Her frasorteres udenlandske annoncer:

In [7]:
#Frasorter de engelske jobannoncer
df['english'] = [0 if 'æ' in x or 'ø' in x or 'å' in x else 2 for x in df['jobbeskrivelse']]

df = df[df.english != 2]


#### Rengør datasæt for forkerte placeringer af data:

In [8]:
# få byttet dem med forkert placering af company og jobbeskrivelse som fx jurist og konsulent, Fuldmægtig
# df[df['company'].str.contains("Jurist")]


# identificer annoncerne med forkerte dataindtastninger under "company"
print(df.loc[df.index==286].company)
print(df.loc[df.index==654].company)
print(df.loc[df.index==239].company)
print(df.loc[df.index==7487].company)
print(df.loc[df.index==9195].company)

# Giv annoncerne det rette virksomhedsnavn
df["company"].replace({"Jurist med interesse for transporterhverv": "Trafikstyrelsen"}, inplace=True)
df["company"].replace({"Jurist/Senior Legal Advisor": "Telia Danmark"}, inplace=True)
df["company"].replace({"Nyuddannet jurist søges til arbejde med lovforslag og ministerbetjening": "Børne- og Undervisningsministeriet"}, inplace=True)
df["company"].replace({"Fuldmægtig/specialkonsulent til alsidige opgaver på søterritoriet": "Miljø- og fødevareministeriet"}, inplace=True)
df["company"].replace({"Fuldmægtige til FødevareErhverv": "Ministeriet for Fødevarer, Landbrug og Fiskeri"}, inplace=True)

# værktøj til at inspicere virksomhedsnavne
df[df['company'].str.contains('jurist')].jobbeskrivelse 


286    Jurist med interesse for transporterhverv
Name: company, dtype: object
654    Jurist/Senior Legal Advisor
Name: company, dtype: object
239    Nyuddannet jurist søges til arbejde med lovfor...
Name: company, dtype: object
7487    Fuldmægtig/specialkonsulent til alsidige opgav...
Name: company, dtype: object
9195    Fuldmægtige til FødevareErhverv
Name: company, dtype: object


431     Gadejuristen, København, søger tre jurister t...
Name: jobbeskrivelse, dtype: object

#### Her kategoriseres offentlige jobannoncer, baseret på navnet på afsendervirksomheden:

In [82]:
# Sorterer offentlige og private jobannoncer fra hinanden
# offentlig er 1 og privat 0

df['offentlig'] = [1 if 'Kommune' in x # først statslige myndigheder
                   or 'kommune' in x
                   or 'ministerie' in x 
                   or 'Ministerie' in x
                   or 'minister' in x
                   or 'styrelse' in x 
                   or 'Styrelse' in x 
                   or 'Region' in x
                   or 'Staten' in x
                   or 'myndighed' in x 
                   or 'Myndighed' in x
                   or 'tilsyn' in x 
                   or 'Politi' in x 
                   or 'politi' in x 
                   or 'Forsvaret'in x
                   or 'DR' in x 
                   or 'Banedanmark' in x
                   or 'Retten' in x
                   or 'Grønland' in x
                   or 'Selvstyre' in x
                   or 'Kriminalforsorgen'in x
                   or 'Statens IT' in x
                   or 'Danmarks Statistik' in x
                   or 'Sundhed.dk' in x
                   or 'Bibliotek' in x
                   or 'SKAT' in x
                   or 'Jobcenter' in x
                   or 'Hjemmeværnskommandoen' in x
                   or 'Statsforvaltningen' in x
                   or 'Nationalbank' in x
                   or 'Vejdirektoratet' in x
                   # under beskæftigelsesministeriet
                   or 'ATP' in x
                   or 'Arbejdsmarkedets Erhvervssikring' in x
                   # herefter uddannelsesinstitutioner
                   or 'Professionshøjskole' in x
                   or 'Universitet'in x
                   or 'Copenhagen Business School' in x
                   or 'University College' in x
                   or 'Erhvervsskolen Nordsjælland' in x
                   or 'Erhvervsakademi' in x
                   or 'UC Syd' in x
                   or 'College' in x 
                   or 'Hotel- og Restaurantskolen' in x
                   or 'Kongelige' in x
                   or 'SIMAC' in x
                   # herefter råd og nævn
                   or 'Det Etiske Råd' in x
                   or 'Patienterstatningen' in x
                   or 'Nævn' in x
                   or 'nævn' in x
                   or 'Den Nationale Klageinstans mod Mobning' in x
                   or 'Rigsadvokaten' in x
                   or 'Rigsombudsmanden i Grønland' in x
                   or 'Den Europæiske Union' in x
                   or 'Nævnenes Hus' in x
                   or 'Dansk Varefakta Nævn' in x
                   # herefter statslige selskaber med fuld aktiemajoritet
                   or 'DSB' in x
                   or 'Vækstfonden' in x
                   or 'EKF Danmarks Eksportkredit' in x
                   or 'Finansiel Stabilitet' in x
                   or 'Investeringsfonden for udviklingslande (IFU)' in x
                   or 'Naviair' in x
                   or 'Evida' in x
                   or 'Sund & Bælt Holding A/S' in x
                   or 'Eksport Kredit Fonden' in x
                   or 'Nationalt Genom Center' in x
                   or 'Københavns Tekniske Skole' in x
                   or 'Energinet' in x
                   or "TV 2" in x
                   or "Nordsøenheden" in x
                   or "Bornholmstrafikken Holding A/S" in x
                   or "Danpilot" in x
                   or "Statens Ejendomssalg A/S" in x
                   or "Det Danske Klasselotteri A/S" in x
                   
                   else 0

for x in df['company']]

# Jeg bliver dog nød til at manuelt korregere fejl, som fx disse firmaer der er blevet klassificeret som offentlig grundet søgning efter "DR".

df.loc[df.company == "DRACHMANN ADVOKATER", 'offentlig'] = 0
df.loc[df.company == "DRACHMANN EJENDOMSADMINISTRATION A/S", 'offentlig'] = 0


#### Her vurderes fordelingen af jobbeskrivelser på år. Jeg laver desuden et alternativt datasæt kun med observationer fra 2021.

In [9]:
# finder fordelingen af annoncer baseret på årstal
print(len(df[df['year'].str.contains("2007")]))
print(len(df[df['year'].str.contains("2008")]))
print(len(df[df['year'].str.contains("2009")]))
print(len(df[df['year'].str.contains("2010")]))
print(len(df[df['year'].str.contains("2011")]))
print(len(df[df['year'].str.contains("2012")]))
print(len(df[df['year'].str.contains("2013")]))
print(len(df[df['year'].str.contains("2014")]))
print(len(df[df['year'].str.contains("2015")]))
print(len(df[df['year'].str.contains("2016")]))
print(len(df[df['year'].str.contains("2017")]))
print(len(df[df['year'].str.contains("2018")]))
print(len(df[df['year'].str.contains("2019")]))
print(len(df[df['year'].str.contains("2020")]))
print(len(df[df['year'].str.contains("2021")]))

# annoncer fra den private og offentlige sektor pr år:
     
print(df[df['year'].str.contains("2007")].offentlig.value_counts())
print(df[df['year'].str.contains("2008")].offentlig.value_counts())
print(df[df['year'].str.contains("2009")].offentlig.value_counts())
print(df[df['year'].str.contains("2010")].offentlig.value_counts())
print(df[df['year'].str.contains("2011")].offentlig.value_counts())
print(df[df['year'].str.contains("2012")].offentlig.value_counts())
print(df[df['year'].str.contains("2013")].offentlig.value_counts())
print(df[df['year'].str.contains("2014")].offentlig.value_counts())
print(df[df['year'].str.contains("2015")].offentlig.value_counts())
print(df[df['year'].str.contains("2016")].offentlig.value_counts())
print(df[df['year'].str.contains("2017")].offentlig.value_counts())
print(df[df['year'].str.contains("2018")].offentlig.value_counts())
print(df[df['year'].str.contains("2019")].offentlig.value_counts())
print(df[df['year'].str.contains("2020")].offentlig.value_counts())
print(df[df['year'].str.contains("2021")].offentlig.value_counts())

242
293
233
310
336
294
327
482
585
777
1067
1306
1278
1289
1540
0    208
1     34
Name: offentlig, dtype: int64
0    245
1     48
Name: offentlig, dtype: int64
0    195
1     38
Name: offentlig, dtype: int64
0    251
1     59
Name: offentlig, dtype: int64
0    261
1     75
Name: offentlig, dtype: int64
0    218
1     76
Name: offentlig, dtype: int64
0    232
1     95
Name: offentlig, dtype: int64
0    323
1    159
Name: offentlig, dtype: int64
0    369
1    216
Name: offentlig, dtype: int64
0    499
1    278
Name: offentlig, dtype: int64
0    625
1    442
Name: offentlig, dtype: int64
0    707
1    599
Name: offentlig, dtype: int64
0    644
1    634
Name: offentlig, dtype: int64
1    673
0    616
Name: offentlig, dtype: int64
1    809
0    731
Name: offentlig, dtype: int64


#### Fordelinger af virksomheder baseret på sektortype:

In [10]:
# Hvor mange virksomheder fremfår fra det private og fra det offentlige?
# HUSK PRIVAT ER 0 og OFFENTLIG ER 1

df['offentlig'].value_counts()

0    6124
1    4235
Name: offentlig, dtype: int64

In [84]:
# opdeling af datasæt i privat og offentlig karakter
offentlig_data = df.loc[df['offentlig'] == 1]
privat_data = df.loc[df['offentlig'] == 0]

# fokus på de pågældende jobbeskrivelser
offentlig_text = offentlig_data.jobbeskrivelse
privat_text = privat_data.jobbeskrivelse

# lav en stor, som ikke adskiller de i sektorer (endnu)
all_text = df.jobbeskrivelse

#### Skaber en liste over alle virksomheder i datasættet - delt op i privat og offentlig:

In [85]:
# numpy.set_printoptions(threshold=sys.maxsize)

#df['company'].unique().tolist()

print("Offentlige virksomheder:")
off_virk = offentlig_data['company'].unique().tolist()
print(len(off_virk))
print("\nPrivate virksomheder:")
priv_virk = privat_data['company'].unique().tolist()
print(len(priv_virk))

print(off_virk)
print("\n")
print("-------- Næste fremkommende er de private virksomheder medtaget i undersøgelsen --------")
print("\n")
print(priv_virk)

Offentlige virksomheder:
294

Private virksomheder:
1333
['Herlev Kommune', 'Skattestyrelsen', 'Region Syddanmark', 'Miljøministeriet', 'Styrelsen for Arbejdsmarked og Rekruttering', 'Energistyrelsen', 'Ministeriet for Fødevarer, Landbrug og Fiskeri', 'Skatteministeriet', 'ATP', 'Konkurrence- og Forbrugerstyrelsen', 'Trafikstyrelsen', 'Banedanmark', 'Færdselsstyrelsen', 'Vurderingsstyrelsen', 'Københavns Universitet', 'Grønlands Selvstyre', 'Vordingborg Kommune', 'Aarhus Business College', 'Erhvervsstyrelsen', 'Socialstyrelsen', 'Finanstilsynet', 'Lyngby-Taarbæk Kommune', 'Roskilde Universitet', 'Arbejdstilsynet', 'Københavns Kommune', 'Styrelsen for Undervisning og Kvalitet', 'Forsvarsministeriets Ejendomsstyrelse,', 'Bolig- og Planstyrelsen', 'Styrelsen For It og Læring', 'Københavns Professionshøjskole', 'IT-Universitetet i København', 'Syd- og Sønderjyllands Politi', 'Vesthimmerlands Kommune', 'Rigspolitiet', 'Domstolsstyrelsen', 'Energinet', 'Uddannelses- og Forskningsministeriet'

In [86]:
# antallet af virksomheder i alt
print(len(off_virk))
print(len(priv_virk))

len(off_virk) + len(priv_virk)

294
1333


1627

#### (1) Her gøres min data til to forskellige sammenhængende lister af ord for hhv. den offentlige og private sektor. 

In [88]:
# skaber den sammenhængende text 


offentlig_sammenhæng_text = " ".join(job for job in offentlig_data.jobbeskrivelse)
privat_sammenhæng_text = " ".join(job for job in privat_data.jobbeskrivelse)

# gør alt tekst til små bogstaver

privat_små = \
     privat_sammenhæng_text.lower()
     #.replace(".","").replace(",","").replace("\n", " ").replace("-")

offentlig_små = offentlig_sammenhæng_text.lower()
    #.replace(".","").replace(",","").replace("\n", " ")

# fjerner tegnsætning og tal

offentlig_u_tegnsætning = offentlig_små.translate(str.maketrans('', '', string.punctuation))
offentlig_clean = offentlig_u_tegnsætning.translate(str.maketrans('', '', string.digits))

privat_u_tegnsætning = privat_små.translate(str.maketrans('', '', string.punctuation))
privat_clean = privat_u_tegnsætning.translate(str.maketrans('', '', string.digits))



In [89]:
# fjerner stopwords
stop_words  =\
    requests\
        .get("https://gist.githubusercontent.com/berteltorp/0cf8a0c7afea7f25ed754f24cfc2467b/raw/305d8e3930cc419e909d49d4b489c9773f75b2d6/stopord.txt")\
        .text \
        .split()


# manuel liste af stopwords
stop_words_manual = ["–", "mv", "”", "’", "bla","'","-"]

stop_words.extend(stop_words_manual)



In [90]:
# her skabes min lemmatizer
lemmatize =\
    lambda word: lemmy\
        .load("da")\
        .lemmatize("", word)[0]

In [92]:
# FÆRDIG MED AT LAVE TO FORSKELLIGE ORD-LISTER UDEN SKELNING TIL UNIKKE JOBANNONCER 

#Her laver du Tokenization og Lemmatization

# Tokenization (og stop words)

privat_tokens = [word for word in nltk.word_tokenize(privat_clean) if word not in stop_words]

offentlig_tokens = [word for word in nltk.word_tokenize(offentlig_clean) if word not in stop_words]

# Lemmatization

privat_done_fluid = [lemmatize(word) for word in privat_tokens]

offentlig_done_fluid = [lemmatize(word) for word in offentlig_tokens]

# finder antal ord indenfor hver sektor
print("Ord fra private organisationers jobbeskrivelser:")
print(len(privat_done_fluid))
print("Ord fra offentlige organisationers jobbeskrivelser:")
print(len(offentlig_done_fluid))

Ord fra private organisationers jobbeskrivelser:
294874
Ord fra offentlige organisationers jobbeskrivelser:
252714


#### (2) Herunder skaber jeg et datasæt hvori de unikke jobannoncer kan identificeres. Dette gør, at jeg fx kan finde en procentandel af brugen af PSM:

In [93]:
# Her skabes en analyserbar version af hele datamaterialet, samlet. 

#gjort små og fjerner \n
text_små = \
     all_text.str.lower().str.replace("\n", " ")

# # fjerner tegnsætning og tal
text_u_tegnsætning = text_små.str.translate(str.maketrans('', '', string.punctuation))
text_clean = text_u_tegnsætning.str.translate(str.maketrans('', '', string.digits))

# # fjerner stop words
text_u_stop = text_clean.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

# # Tokenization
text_tokens = text_u_stop.apply(nltk.word_tokenize)

# # Lemmatization 
text_done = text_tokens.apply(lambda lst:[lemmatize(word) for word in lst])

df_done = pd.concat([df.company, text_done, df.offentlig], axis=1)
           

In [94]:
# deler vores df_done op i hhv. private og offentlige beskrivelser

privat_done = df_done[df_done["offentlig"]==0]
offentlig_done = df_done[df_done["offentlig"]==1]


In [95]:
# den fulde df uden ovenstående sample
# df_done_final = df_done[~df_done.index.isin(df_sample.index)] OBS - DETTE ER GJORT FOROVEN

samlet_antal_off = len(df.loc[df['offentlig'] == 1])
samlet_antal_priv = len(df.loc[df['offentlig'] == 0])


### Søge sekventielt efter forekomsten af ord (PSM-indikatorer): 

In [96]:
# Vi vil gerne kunne se alle ord i cellerne
pd.set_option('display.max_colwidth', None)


#### Attraction to public policy making:

In [97]:
# Attraction to public policy making
# fortolke dele

#overvej om denne dimemsion er lidt for bredt defineret i din ordbog

verber_adjektiver_atp = "skabe|fortolke|dele|tilhøre|forbinde|tilfredsstille|gavn|involvere|interessere|politisk|attraktiv|bryde|lide|høj|positiv"
substantiver_atp = "lov|land|fællesskab|holdning|program|gavn|menneske|politik|politisk|spil|tanke"


#Den første søgning efter verber og adjektiver
verber_adjektiver_atp_off = offentlig_done[offentlig_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_atp)]
verber_adjektiver_atp_priv = privat_done[privat_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_atp)]

#sekventiel søgning efter substantiver
alle_ordklasser_atp_off = verber_adjektiver_atp_off[verber_adjektiver_atp_off['jobbeskrivelse'].astype(str).str.contains(substantiver_atp)]
alle_ordklasser_atp_priv = verber_adjektiver_atp_priv[verber_adjektiver_atp_priv['jobbeskrivelse'].astype(str).str.contains(substantiver_atp)]

#udregner den offentlige andel
quotient = len(alle_ordklasser_atp_off) / samlet_antal_off 

percent = quotient * 100

print("Den offentlige procentandel:")
print(percent)

#udregner den private andel
quotient = len(alle_ordklasser_atp_priv) / samlet_antal_priv

percent = quotient * 100

print("Den private procentandel:")
print(percent)

# her indsættes attraction to public policy i df'en som binær-variabel
atp_index = [i for i in alle_ordklasser_atp_off.index]
atp_index.extend([i for i in alle_ordklasser_atp_priv.index])

df_done['ATP'] = df_done.index.map(lambda x : 1 if x in atp_index else 0)

Den offentlige procentandel:
45.21664521664522
Den private procentandel:
24.995611725469548


#### Commitment to the public interest:

In [98]:

verber_adjektiver_cpi = "bidrage|gavne|tjene|god|meningsfuld|vigtig|uselvisk|fælles|hel"
substantiver_cpi = "borger|pligt|samfund|dyd|interesse|ydelse|ansat"

# Den første søgning efter verber og adjektiver blandt hhv. offentlig og private jobbeskrivelser
verber_adjektiver_cpi_off = offentlig_done[offentlig_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_cpi)]
verber_adjektiver_cpi_priv = privat_done[privat_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_cpi)]

# sekventiel søgning efter substantiver i det netop fremsøgte materiale
alle_ordklasser_cpi_off = verber_adjektiver_cpi_off[verber_adjektiver_cpi_off['jobbeskrivelse'].astype(str).str.contains(substantiver_cpi)]
alle_ordklasser_cpi_priv = verber_adjektiver_cpi_priv[verber_adjektiver_cpi_priv['jobbeskrivelse'].astype(str).str.contains(substantiver_cpi)]

#udregner den offentlige andel
quotient = len(alle_ordklasser_cpi_off) / samlet_antal_off 

percent = quotient * 100

print("Den offentlige procentandel:")
print(percent)

#udregner den private andel
quotient = len(alle_ordklasser_cpi_priv) / samlet_antal_priv

percent = quotient * 100

print("Den private procentandel:")
print(percent)

# her indsættes commitment to public interest i datasættet som binær variabel
cpi_index = [i for i in alle_ordklasser_cpi_off.index]
cpi_index.extend([i for i in alle_ordklasser_cpi_priv.index])

df_done['CPI'] = df_done.index.map(lambda x : 1 if x in cpi_index else 0)

# df_done[df_done['CPI']==1]

Den offentlige procentandel:
19.81981981981982
Den private procentandel:
9.461119887660171


#### Compassion: 

In [99]:

verber_adjektiver_com = "blive|føle|behandle|følelsesmæssig|berørt|følsom|afhængig|retfærdig|mindre"
substantiver_com = "nød|sympati|afhængig|menneske|hensyn|velfærd|problem|fremmed|privilegie"

# Den første søgning efter verber og adjektiver blandt hhv. offentlig og private jobbeskrivelser
verber_adjektiver_com_off = offentlig_done[offentlig_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_com)]
verber_adjektiver_com_priv = privat_done[privat_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_com)]

# sekventiel søgning efter substantiver i det netop fremsøgte materiale
alle_ordklasser_com_off = verber_adjektiver_com_off[verber_adjektiver_com_off['jobbeskrivelse'].astype(str).str.contains(substantiver_com)]
alle_ordklasser_com_priv = verber_adjektiver_com_priv[verber_adjektiver_com_priv['jobbeskrivelse'].astype(str).str.contains(substantiver_com)]

#udregner den offentlige andel
quotient = len(alle_ordklasser_com_off) / samlet_antal_off 

percent = quotient * 100

print("Den offentlige procentandel:")
print(percent)

#udregner den private andel
quotient = len(alle_ordklasser_com_priv) / samlet_antal_priv

percent = quotient * 100

print("Den private procentandel:")
print(percent)

# her indsættes commitment to public interest i datasættet som binær variabel
com_index = [i for i in alle_ordklasser_com_off.index]
com_index.extend([i for i in alle_ordklasser_com_priv.index])

df_done['COM'] = df_done.index.map(lambda x : 1 if x in com_index else 0)

# df_done[df_done['COM']==1]

Den offentlige procentandel:
6.263406263406264
Den private procentandel:
2.9313673863436898


#### Self-sacrifice:

In [100]:
# self-sacrifice

verber_adjektiver_ss = "gøre|give|yde|risikere|hjælpe|tilsidesætte|tjene|ofre|bidrage|god|personlig|uselvisk"
substantiver_ss = "forskel|pligt|gerning|samfund|skyld|tab|behov|borger|pligt|afsavn|hensyn|følelse"

# til analyse : overvej, at disse ord også kan være relevante at signalere for private virksomheder (e.g. de skal også give mening til ansatte), der er den så høj

# Den første søgning efter verber og adjektiver blandt hhv. offentlig og private jobbeskrivelser
verber_adjektiver_ss_off = offentlig_done[offentlig_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_ss)]
verber_adjektiver_ss_priv = privat_done[privat_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_ss)]

# sekventiel søgning efter substantiver i det netop fremsøgte materiale
alle_ordklasser_ss_off = verber_adjektiver_ss_off[verber_adjektiver_ss_off['jobbeskrivelse'].astype(str).str.contains(substantiver_ss)]
alle_ordklasser_ss_priv = verber_adjektiver_ss_priv[verber_adjektiver_ss_priv['jobbeskrivelse'].astype(str).str.contains(substantiver_ss)]

# udregner den offentlige andel
quotient = len(alle_ordklasser_ss_off) / samlet_antal_off 

percent = quotient * 100

print("Den offentlige procentandel:")
print(percent)

#udregner den private andel
quotient = len(alle_ordklasser_ss_priv) / samlet_antal_priv

percent = quotient * 100

print("Den private procentandel:")
print(percent)

# her indsættes commitment to public interest i datasættet som binær variabel
ss_index = [i for i in alle_ordklasser_ss_off.index]
ss_index.extend([i for i in alle_ordklasser_ss_priv.index])

df_done['SS'] = df_done.index.map(lambda x : 1 if x in ss_index else 0)

# df_done[df_done['SS']==1]


Den offentlige procentandel:
30.60918060918061
Den private procentandel:
19.747235387045812


### Signifikanstest af ovenstående forskel:

In [61]:
# t-test - hvis hver "psm-indikator" markeres med en kolonne, med enten 0 eller 1

from scipy.stats import ttest_ind

off = df_done[df_done['offentlig']==1]
priv = df_done[df_done['offentlig']==0]

In [46]:
# attraction to public policy, t-test

ttest_atp = ttest_ind(off['ATP'], priv['ATP']).pvalue

output_atp = f"{ttest_atp:.50f}"
print(output_atp)


0.00000000000000000000000000000000000000000000000000


In [47]:
# commitment to public interest, t-test

ttest_cpi = ttest_ind(off['CPI'], priv['CPI']).pvalue

output_cpi = f"{ttest_cpi:.50f}"

print(output_cpi)

0.00000000000000000000000000000000000000000000001903


In [48]:
# compassion, t-test

ttest_com = ttest_ind(off['COM'], priv['COM']).pvalue

output_com = f"{ttest_com:.50f}"

print(output_com)


0.00000000000023606094513038132098556729756569103952


In [60]:
# self-sacrifice, t-test

ttest_ss = ttest_ind(off['SS'], priv['SS']).pvalue

output_ss = f"{ttest_ss:.50f}"

print(output_ss)

0.00000000000000000000000000000019468582947709186240


### Forsøg på at finde ekstrinsiske indikatorer:

##### Ydre motivation: materiel

In [101]:
ydre_mot_verber_adj = "tilbyde|attraktiv|høj|konkurrencedygtig|stor|god"
ydre_mot_sub = "løn|personalegode|bonus"

# offentlig_done[offentlig_done['jobbeskrivelse'].astype(str).str.contains("løn|personalegode|bonus")]
# privat_done[privat_done['jobbeskrivelse'].astype(str).str.contains("tillæg|vederlag")]

In [104]:
# først søger jeg efter verber og adjektiver

priv_mot_verber_adj = privat_done[privat_done['jobbeskrivelse'].astype(str).str.contains(ydre_mot_verber_adj)]
off_mot_verber_adj = offentlig_done[offentlig_done['jobbeskrivelse'].astype(str).str.contains(ydre_mot_verber_adj)]

# herefter søger jeg efter substantiver

priv_ydre_mot_mate = priv_mot_verber_adj[priv_mot_verber_adj['jobbeskrivelse'].astype(str).str.contains(ydre_mot_sub)]
off_ydre_mot_mate = off_mot_verber_adj[off_mot_verber_adj['jobbeskrivelse'].astype(str).str.contains(ydre_mot_sub)]


# udregner den offentlige andel
quotient = len(off_ydre_mot_mate) / samlet_antal_off 

percent = quotient * 100

print("Den offentlige procentandel:")
print(percent)

#udregner den private andel
quotient = len(priv_ydre_mot_mate) / samlet_antal_priv

percent = quotient * 100

print("Den private procentandel:")
print(percent)


# her indsættes monetært incitament i datasættet som binær variabel
mi_index = [i for i in off_ydre_mot_mate.index]
mi_index.extend([i for i in priv_ydre_mot_mate.index])

df_done['MI'] = df_done.index.map(lambda x : 1 if x in mi_index else 0)



Den offentlige procentandel:
1.522951522951523
Den private procentandel:
3.598385114972793


In [65]:
# monetært incitament, t-test

ttest_øi = ttest_ind(off['ØI'], priv['ØI']).pvalue

output_øi = f"{ttest_øi:.50f}"

print(output_øi)

0.00000000000962318588658014684851984598159527119904


In [103]:
# Opsummerende statistik

summary = [len(x) for x in df_done.jobbeskrivelse]

mySeries = pd.Series(summary) 

print("Jobbeskrivelserne:")
print(mySeries.describe())

print("\nResten af variablerne:")
print(df_done.describe())

Jobbeskrivelserne:
count    10359.000000
mean        53.043923
std         10.827587
min          8.000000
25%         47.000000
50%         54.000000
75%         61.000000
max        101.000000
dtype: float64

Resten af variablerne:
          offentlig           ATP           CPI           COM            SS  \
count  10359.000000  10359.000000  10359.000000  10359.000000  10359.000000   
mean       0.450043      0.340960      0.141230      0.044309      0.246356   
std        0.497522      0.474055      0.348275      0.205791      0.430909   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        1.000000      1.000000      0.000000      0.000000      0.000000   
max        1.000000      1.000000      1.000000      1.000000      1.000000   

                 ØI  
count  10359.000000  
mean      